In [ ]:
# This code is for fine-tuning the CER dataset with BioMistral-7B.
# After fine-tuning, the resulting checkpoint can be used as a preprocessing step to resolve coordination ellipses in the text column.
# Our version of CER is available at: https://drive.google.com/drive/folders/1qmeKo0cR3oLdAG5Ov8fqxqv2IwByAEBe?usp=share_link

In [ ]:
# This code incorporates parts from:
#https://www.kaggle.com/code/lucamassaron/fine-tune-mistral-v0-2-for-sentiment-analysis
#https://ubiai.tools/fine-tuning-mistral-for-ner/

In [ ]:
pip install -q -U torch=='2.0.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install -q -U accelerate=='0.25.0' peft=='0.7.1' bitsandbytes=='0.41.3.post2' transformers=='4.36.1' trl=='0.7.4'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging,
                          TextStreamer)
from huggingface_hub import notebook_login
#base_model = "mistralai/Mistral-7B-v0.1"
#base_model = "BioMistral/BioMistral-7B"
base_model = "BioMistral/BioMistral-7B-DARE"

Dataset

In [ ]:
import pandas as pd

Training_path = "/content/drive/MyDrive/DocDiscNER/CompoundResolverDataset/ExpandedCompoundResolverDataset.csv"
Validation_path = "/content/drive/MyDrive/DocDiscNER/CompoundResolverDataset/ValidPosCompoundResolverDataset.csv"
Test_path = "/content/drive/MyDrive/DocDiscNER/CompoundResolverDataset/CADEC/CADEC_Test.csv"

Training = pd.read_csv(Training_path, names=["Input", "Target"], encoding="utf-8", skiprows=1)
Validation = pd.read_csv(Validation_path, names=["Input", "Target"], encoding="utf-8",skiprows=1)
Test = pd.read_csv(Test_path, names=["Input", "Target"], encoding="utf-8",skiprows=1)

In [ ]:
def generate_prompt(data_point):
    return f"""
            Input:
            [{data_point["Input"]}]
            Target:
            {data_point["Target"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Input:
            [{data_point["Input"]}]
            Target:""".strip()

In [ ]:
Training = pd.DataFrame(Training.apply(generate_prompt, axis=1), columns=["Input"])
Validation = pd.DataFrame(Validation.apply(generate_prompt, axis=1), columns=["Input"])
Test = pd.DataFrame(Test.apply(generate_test_prompt, axis=1), columns=["Input"])

train_data = Dataset.from_pandas(Training)
val_data = Dataset.from_pandas(Validation)
test_data = Dataset.from_pandas(Test)

In [ ]:
print(test_data[0])

{'Input': 'Input:\n            [weight gain, confusion, depression.\nSTOP Taking LIPITOR.\nThis drug will hurt you more than help you.\nPfizer should be sued.\n]\n            Target:'}


Model

In [ ]:
# Load base model(Mistral 2B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, token=os.environ['HF_TOKEN'])
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_eos_token = True
model.config.pad_token_id = tokenizer.pad_token_id


#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=256,
        lora_alpha=128,
        lora_dropout=0.1, #the Conventional=0.05
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=5e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="linear",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=peft_config,
    dataset_text_field="Input",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=512,
)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [ ]:
# Monitering the LLM
wandb.login(key = "3be55aa2732d4d435809cd33149e996e24b470d8")
run = wandb.init(project='Fine tuning mistral 7B UBIAI', job_type="training", anonymous="allow")

In [ ]:
import torch
torch.cuda.empty_cache()
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,No log,1.210932
2,No log,0.647404
4,No log,0.265432
6,No log,0.080531
8,No log,0.071129
10,0.372300,0.072034
12,0.372300,0.072445
14,0.372300,0.072968
16,0.372300,0.072994


TrainOutput(global_step=40, training_loss=0.2416442319750786, metrics={'train_runtime': 471.3836, 'train_samples_per_second': 6.789, 'train_steps_per_second': 0.085, 'total_flos': 3.758489064013824e+16, 'train_loss': 0.2416442319750786, 'epoch': 16.0})

In [ ]:
wandb.finish()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/runs

In [ ]:
trainer.save_model("/content/drive/MyDrive/DocDiscNER/CompoundResolverDataset/Checkpoints/CER_BioMistral7B")

In [ ]:
model_path = "/content/drive/MyDrive/DocDiscNER/CompoundResolverDataset/Checkpoints/CER_BioMistral7B"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path,
      quantization_config=bnb_config,
      device_map={"": 0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
############### PREDICT ###############

In [ ]:
def predict(X_test, model, tokenizer):
    prompt = X_test
    pipe = pipeline(task="text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens = 800,
                    temperature = 0.0,
                    )
    result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
    answer = result[0]['generated_text'].split("Target:")[-1].strip()

    return answer

In [ ]:
# Trying one example
output = predict(test_data[6]["Input"], model, tokenizer)
print(output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


dosage was increased from 10 to 20 mg and the tingling in my feet and numbness in my feet and tingling in my shins and numbness in my shins escalated.
I even went to a podiatrist because my toes ached and my feet hurt.
I've tried simvastatin, provastatin, and lovastatin.
Each time the side effects return.
I can't even sleep at night because of the sensations of tingling in my legs and tingling in my feet and sensations of numbness in my legs and sensations of numbness in my feet.
No more statins for me.
Does anyone else have the pain in their toes, pain in their feet and pain in their shins?.


In [ ]:
# Iterate over the All test examples
outputs = []

#for data in test_data:
num_examples = len(test_data)
for i in range(num_examples):
    output = predict(test_data[i]["Input"], model, tokenizer)
    #output = generate_response(data["Text"], model)
    outputs.append(output)

In [ ]:
# write in file
import csv
Txt = []
Tgt = []
# Open a new TSV file for writing

with open("/content/drive/MyDrive/DocDiscNER/CompoundResolverDataset/CADEC/CADEC_Test.csv", 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)
        for row in reader:
            Txt.append(row[0])
            Tgt.append(row[1])
data = list(zip(Txt, outputs, Tgt))
file_path = '/content/drive/MyDrive/DocDiscNER/CompoundResolverDataset/Resolved_CADEC_Test.tsv'

with open(file_path, "w", newline="", encoding="utf-8") as tsvfile:
    writer = csv.writer(tsvfile, delimiter="\t")
    writer.writerow(["Text", "Output", "Target"])  # Write the header row
    writer.writerows(data)  # Write the data rows

In [ ]:
from google.colab import runtime
runtime.unassign()